# Assignment 2 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


A courier company operates a fleet of delivery trucks that make deliveries to different parts of the city. The trucks are equipped with GPS tracking devices that record the location of each truck at regular intervals. The locations are divided into three regions: downtown, the suburbs, and the countryside. The following table shows the probabilities of a truck transitioning between these regions at each time step:

| Current region | Probability of transitioning to downtown | Probability of transitioning to the suburbs | Probability of transitioning to the countryside |
|----------------|--------------------------------------------|-----------------------------------------------|------------------------------------------------|
| Downtown       | 0.3                                      | 0.4                                           | 0.3                                            |
| Suburbs        | 0.2                                      | 0.5                                           | 0.3                                            |
| Countryside    | 0.4                                      | 0.3                                           | 0.3                                            |

1. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region after two time steps? [1.5p]
2. If a truck is currently in the suburbs, what is the probability that it will be in the downtown region **the first time** after two time steps? [1.5p]
3. Is this Markov chain irreducible? [1.5p]
4. What is the stationary distribution? [1.5p]
5. Advanced question: What is the expected number of steps until the first time one enters the downtown region having started in the suburbs region. Hint: to get within 1 decimal point, it is enough to compute the probabilities for hitting times below 30. [2p]



In [21]:
# Part 1

# Fill in the answer to part 1 below as a decimal number
problem1_p1 = 0.28

In [22]:
# Part 2

# Fill in the answer to part 2 below as a decimal number
problem1_p2 = 0.22

In [23]:
# Part 3

# Fill in the answer to part 3 below as a boolean
problem1_irreducible = True

In [24]:
# Part 4

# Fill in the answer to part 4 below
# the answer should be a numpy array of length 3
# make sure that the entries sums to 1!
problem1_stationary =  [0.28888889,0.41111111,0.3]

In [25]:
# Part 5

# Fill in the answer to part 5 below
# That is, the expected number of steps as a decimal number
problem1_ET = 3.846

---
## Assignment 2, PROBLEM 2
Maximum Points = 8


A healthcare organization is interested in understanding the relationship between the number of visits to the doctors office and certain patient characteristics. 
They have collected data on the number of visits (for a year) for a sample of patients and have included the following variables

* ofp : number of physician office visits
* ofnp : number of nonphysician office visits
* opp : number of physician outpatient visits
* opnp : number of nonphysician outpatient visits
* emr : number of emergency room visits
* hosp : number of hospitalizations
* exclhlth : the person is of excellent health (self-perceived)
* poorhealth : the person is of poor health (self-perceived)
* numchron : number of chronic conditions
* adldiff : the person has a condition that limits activities of daily living ?
* noreast : the person is from the north east region
* midwest : the person is from the midwest region
* west : the person is from the west region
* age : age in years (divided by 10)
* male : is the person male ?
* married : is the person married ?
* school : number of years of education
* faminc : family income in 10000$
* employed : is the person employed ?
* privins : is the person covered by private health insurance?
* medicaid : is the person covered by medicaid ?

Decide which patient features are resonable to use to predict the target "number of physician office visits". Hint: should we really use the "ofnp" etc variables?

Since the target variable is counts, a reasonable loss function is to consider the target variable as Poisson distributed where the parameter follows $\lambda = \exp(\alpha \cdot x + \beta)$ where $\alpha$ is a vector (slope) and $\beta$ is a number (intercept). That is, the parameter is the exponential of a linear function. The reason we chose this as our parameter, is that it is always positive which is when the Poisson distribution is defined. To be specific we make the following assumption about our conditional density of $Y \mid X$,
$$
    f_{Y \mid X} (y,x) = \frac{\lambda^{y} e^{-\lambda}}{y !}, \quad \lambda(x) = \exp(\alpha \cdot x + \beta).
$$

Recall from the lecture notes, (4.2) that in this case we should consider the log-loss (entropy) and that according to (4.2.1 Maximum Likelihood and regression) we can consider the conditional log-likelihood. Follow the steps of Example 1 and Example 2 in section (4.2) to derive the loss that needs to be minimized.

Hint: when taking the log of the conditional density you will find that the term that contains the $y!$ does not depend on $\lambda$ and as such does not depend on $\alpha,\beta$, it can thus be discarded. This will be essential due to numerical issues with factorials.

Instructions:

1. Load the file `data/visits_clean.csv`, follow the instructions in the code cell of how this should happen [1.5p]
2. Create the `problem2_X` and the `problem2_y` as numpy arrays with `problem2_X` being the features and `problem2_y` being the target. Do the standard train-test split with 80% training data and 20% testing data. Store these in the variables defined in the cells. [1.5p]
3. Implement $loss$ inside the class `PoissonRegression` by writing down the loss to be minimized, I have provided a formula for the $\lambda$ that you can use. [1.5p]
4. Now use the `PoissonRegression` class to train a Poisson regression model on the training data. [1.5p]
5. Compute the mean absolute error of your prediction on the test set and use Hoeffdings inequality to produce a 95\% confidence interval for the mean absolute error. We can make the assumption that the error is bounded by 70 for simplicity. [2p]


In [34]:
# Part 1

# As in assignment 1 we will load the header into header and data into data
# this time you will have to parse the data such that each data entry is a float
# and that the problem2_data is a numpy array of shape (n_samples,n_columns)
# where n_columns is the number of columns and should have the same length as
# the list of strings header. n_samples is how many rows of data we had.
# If you cannot find the file, check the starting package as it should be updated
# if not, go to the github repo and pull it
#合理的患者特征包括exclhlth, poorhealth, numchron, adldiff, age, male, married, school, faminc, employed, privins, medicaid，
#不合理的特征包括ofnp, opp, opnp, emr, hosp，这些特征本身就是就诊次数的不同类型，不能用来预测ofp。
#使用numpy库加载数据文件，使用train_test_split函数划分训练集和测试集，使用PoissonRegression类定义模型并拟合训练数据，使用predict函数预测测试数据，并使用mean_absolute_error函数计算误差和置信区间。
# The autograder does not accept pandas as a solution to this problem.
# data/visits_clean.csv
import csv
import numpy as np
from scipy import optimize
import scipy.special
data_row=[]
# correct_feature=["ofp","exclhlth", "poorhlth", "numchron", "adldiff", "age", "male", "married", "school", "faminc"\
    # , "employed", "privins", "medicaid"]
# correct_feature1=['ofp','numchron','adldiff','noreast','midwest','west','age','male',
#                  'married','school','faminc','employed','privins','medicaid']
problem2_header=[]
# with open("data/visits_clean.csv","r") as file:
with open(r"data/visits_clean.csv","r") as file:
    reader = csv.reader(file)
    problem2_header = next(reader)[0].strip().split(" ")
    
    #numpy.genfromtxt(fname, dtype=<type 'float'>, comments='#', delimiter=None, skip_header=0, skip_footer=0, converters=None, missing_values=None, filling_values=None, usecols=None, names=None, excludelist=None, deletechars=None, replace_space='_', autostrip=False, case_sensitive=True, defaultfmt='f%i', unpack=None, usemask=False, loose=True, invalid_raise=True, max_rows=None)
    # for row in reader:
    #     data_row=[float(eval(value)) for value in row]
    #     data_rows.append(data_row)

# feature_index=[problem2_header.index(x) for x in correct_feature]
# geography_index=[problem2_header.index(x) for x in ["noreast","midwest","west"]]

data_rows=np.genfromtxt(r"data/visits_clean.csv",dtype=float,delimiter=" ",skip_header=1)
# data_rows=np.genfromtxt(r"data/visits_clean.csv",dtype=float,delimiter=" ",skip_header=1,\
#                usecols=feature_index)
# geography=np.genfromtxt(r"data/visits_clean.csv",dtype=float,delimiter=" ",skip_header=1,\
#                 usecols=geography_index)
# #problem2_header = [""] #List of strings

#+1dataset,combine three geography features in to one onehotcode for test Xtrain 
problem2_data = np.array(data_rows) #A numpy array of shape n_samples n_columns
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(problem2_data)
problem2_data = scaler.transform(problem2_data) 


geography_new=np.zeros((len(geography),1))
for i,num in zip(geography,range(len(geography))):
   
    if i[0]==1:
        geography_new[num]=1
     
    elif i[1]==1:
        geography_new[num]=2
  
    elif i[2]==1:
        geography_new[num]=3
       
    else:
        geography_new[num]=4
        
    
from sklearn import preprocessing
encoder = preprocessing.OneHotEncoder()
encoder.fit(geography_new)
geography_onehot=encoder.transform(geography_new).toarray()
# problem2_data=np.hstack((problem2_data,geography_onehot))
# Part 2

# Fill in your X and y below +3数据集，仅用于测试总数据集形状
# feature_index1=[problem2_header.index(x) for x in correct_feature]
# data_rows1=np.genfromtxt(r"data/visits_clean.csv",dtype=float,delimiter=" ",skip_header=1,\
#                usecols=feature_index1)
# problem2_data1 = np.array(data_rows)
problem2_X = problem2_data[[],1:]
problem2_y = problem2_data[[],0]

indices = np.arange(problem2_data.shape[0])
np.random.seed(0)
np.random.shuffle(indices)

train_ratio=0.8
total_samples=problem2_data.shape[0]
train_samples=int(total_samples*train_ratio)

problem2_X_train = problem2_data[indices[:train_samples],1:]
problem2_y_train = problem2_data[indices[:train_samples],0]


problem2_X_test = problem2_data[indices[train_samples:], 1:]
problem2_y_test = problem2_data[indices[train_samples:], 0]
# Split the data problem2_data into train and randomly using for instance
# np.random.shuffle indices and indexing the first 80% as the train data
# keep the train size as 0.8 rounded up to the nearest integer sample

print(problem2_X_train.shape,problem2_y_train.shape)

# Part 3

# Fill in the function loss below

class PoissonRegression(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        # The parameter lambda for the given X and the proposed values 
        # of the coefficients, here coeff[:-1] represent alpha 
        # and coeff[-1] represent beta
        lam = np.exp(np.dot(X, coeffs[:-1]) + coeffs[-1])

        # use the Y variable that is available here to define 
        # the loss function, return the value of the loss for 
        # this Y and for this parameter lam defined above
        neg_log_likelihood = -np.sum(Y * np.log(lam) - lam)
        # neg_log_likelihood=-np.sum(Y*np.log(lam)-lam-np.log(scipy.special.factorial(Y)))
        return neg_log_likelihood

    def fit(self,X,Y):


        #Use the loss above together with an optimization method from scipy
        #to find the coefficients of the model
        #this is prepared for you below
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1) # initial guess as 0
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x
    
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            return np.exp(np.dot(X,self.coeffs[:-1])+self.coeffs[-1])
        
# Part 4

# Initialize your PoissonRegression model
problem2_model = PoissonRegression()

# Fit your initialized model on the training data
problem2_model.fit(problem2_X_train,problem2_y_train)

# This is to make sure that everything went well, 
# check that success is True
print(problem2_model.result)


# Part 5

# Put the computed mean absolute error in the variable below
from sklearn.metrics import mean_absolute_error

# Assuming you have predictions and actual values for the test data
predictions = problem2_model.predict(problem2_X_test)
actual_values = problem2_y_test

# Calculate MAE
MAE = mean_absolute_error(actual_values, predictions)

problem2_metric = MAE
# Put a confidence interval in the variable below by using Hoeffdings inequality using the bounds
# a = 0, b=70 (roughly 5 days between visits as minimum)
# the variable should contain a tuple representing the confidence interval of the form (l_edge,r_edge)

# Calculate the confidence interval using Hoeffding's inequality
a = 0  # Lower bound
b = 70  # Upper bound

# Calculate the confidence interval
# epsilon = np.sqrt((1 / (2 * len(problem2_X_test)) * np.log(2 / 0.05))) 
epsilon =np.sqrt((b - a) ** 2 * np.log(2 / 0.05) / (2 * len(problem2_y_test)))
l_edge = max(MAE - epsilon,a)
r_edge =min(MAE + epsilon,b)
 
# Create a tuple representing the confidence interval
problem2_interval = (l_edge, r_edge)

(3524, 20) (3524,)
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 831.3003007886315
       x: [ 2.080e+00  2.514e-01 ...  2.022e-01 -3.583e+00]
     nit: 412
     jac: [ 0.000e+00  7.629e-06 ... -7.629e-06  0.000e+00]
    nfev: 17712
    njev: 805


In [35]:
MAE

0.04569161802251758

In [28]:
# import numpy as np
# import pandas as pd
# from sklearn.linear_model import PoissonRegressor

# # 定义模型
# model = PoissonRegressor()

# # 训练模型
# model.fit(problem2_X_train, problem2_y_train)

# # 预测结果
# y_pred = model.predict(problem2_X_test)
# # 打印参数
# print(model.coef_)

# # 打印
# MAE=mean_absolute_error(problem2_y_test,y_pred)
# MAE

---
## Assignment 2, PROBLEM 3
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [29]:

def problem3_LCG(size=None, seed=0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e., u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    
    a = 1664525
    b = 1013904223
    M = 2**32  # A large M value
    state = seed
    
    # Initialize the state (seed)
    state = seed
    
    # List to hold the pseudorandom numbers
    out = []
    
    # Generate pseudorandom numbers up to the specified size
    for _ in range(size if size is not None else 1):
        # Update the state using the LCG formula
        state = (a * state + b) % M
        # Append the state to the list after normalizing
        out.append(state / M)
    
    return out

In [30]:

def problem3_uniform(generator=None, period=1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size, seed) that produces pseudo random numbers
                in the range {0,1,...,period-1}
    period    : the period of the generator
    seed      : the seed to be used in the generator provided
    size      : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    # Generate pseudo random numbers with the given generator
    random_numbers = generator(size=size, seed=seed)
    
    # Normalize these numbers to the range [0,1]
    uniform_random_numbers = [number / period for number in random_numbers]
    
    return uniform_random_numbers

In [31]:
def problem3_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    # No need to set np.random.seed here since we're not using np.random directly
    
    # Initialize a variable to keep track of the current state.
    current_seed = seed
    samples = []
    M = np.pi / 2  # The bound for the accept-reject criterion

    while len(samples) < n_iterations:
        # Sample from the uniform distribution using the provided generator
        x = uniformGenerator(size=1, seed=current_seed)[0]
        
        # Calculate the probability of accepting the sample
        f_x = (np.pi / 2) * abs(np.sin(x * 2 * np.pi))

        # Use the next seed value for the acceptance check
        current_seed += 1
        u = uniformGenerator(size=1, seed=current_seed)[0]  # Uniform random number for acceptance

        # Increment the seed for the next iteration
        current_seed += 1
        
        # Accept the sample with probability f(x)/(M*g(x))
        if u < f_x / M:
            samples.append(x)
    
    return samples

# You can then call your function with the initial seed:
print("Accept-Reject sampler %s" % problem3_accept_reject(uniformGenerator=uniform_sampler, n_iterations=20, seed=1))

---
#### Local Test for Assignment 2, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [32]:

print("LCG output: %s" % problem3_LCG(size=10, seed=1))

# Set the period to the modulus used in your LCG (replace 2**31 - 1 with your modulus if different)
period = 2**31 - 1

# Print the output of the uniform sampler
print("Uniform sampler %s" % problem3_uniform(generator=problem3_LCG, period=period, size=10, seed=1))

# Define the uniform_sampler function using a lambda expression
uniform_sampler = lambda size, seed: problem3_uniform(generator=problem3_LCG, period=period, size=size, seed=seed)

LCG output: [7.826369259425611e-06, 0.13153778814316625, 0.7556053221950332, 0.4586501319234493, 0.5327672374121692, 0.21895918632809036, 0.04704461621448613, 0.678864716868319, 0.6792964058366122, 0.9346928959408276]
Uniform sampler [3.644437185986921e-15, 6.125205578488219e-11, 3.51856147193764e-10, 2.1357561095479172e-10, 2.4808907772426417e-10, 1.0196081662087291e-10, 2.190685655753733e-11, 3.1612101811191066e-10, 3.163220389526963e-10, 4.3525029736388376e-10]


In [33]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem3_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.34681819919543666, 0.6840807597768621, 0.7112329216829939, 0.4472134383484645, 0.4306555016160535, 0.7362574515465967, 0.8064626626086779, 0.3982697025447973, 0.9070912163239052, 0.09034095270802844, 0.7793670805766362, 0.30161521354562826, 0.3506308489160106, 0.742923087744433, 0.5808862932227294, 0.6420041783046236, 0.03564764807021392, 0.7818439804824748, 0.7938147365943296, 0.8392775084388612]
